In [ ]:
import numpy as np
from copy import deepcopy
from collections import deque


In [ ]:
pip install jdc


In [ ]:
import jdc #need pip install jdc to run it
#use do have jupiter dynamic classes to avoid copy paste for the class

### 1. Formulation du problème :


In [ ]:
class EightPuzzle:
    def __init__(self, initial_state, goal_state): #basic constructor with initial and goal state
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.taille = 3 #we declare the size off the row or column to use it after so its easier

    def empty_tile(self, state):
        return tuple(np.argwhere(state == 0)[0])

    def move_possible(self, state):
        empty_tile = self.empty_tile(state) #get the 0 tile
        x, y = empty_tile
        moves = []
        if x > 0: #if the place up is not a "wall"
            moves.append("up")  #we can go up then
        if x < self.taille - 1:#if the bottom tile is available we can go down
            moves.append("down")
        if y > 0:# we check if the left tile is available
            moves.append("left")
        if y < self.taille - 1:#we check is right is available
            moves.append("right")
        return moves

    def move(self, state, action):#in this method we've just store a tile in a variable and we change it with the 0
        empty_tile = self.empty_tile(state)
        x, y = empty_tile

        new_state = deepcopy(state)

        if action == "up" and x > 0:#all of these are for each move the tile is replaced
            new_state[x, y], new_state[x - 1, y] = new_state[x - 1, y], new_state[x, y]
        elif action == "down" and x < self.taille - 1:
            new_state[x, y], new_state[x + 1, y] = new_state[x + 1, y], new_state[x, y]
        elif action == "left" and y > 0:
            new_state[x, y], new_state[x, y - 1] = new_state[x, y - 1], new_state[x, y]
        elif action == "right" and y < self.taille - 1:
            new_state[x, y], new_state[x, y + 1] = new_state[x, y + 1], new_state[x, y]

        return new_state


    def is_goal(self, state): #checking if the state we have is the goal state
        return np.array_equal(state, self.goal_state)

    def state_to_tuple(self, state): #modifiying array to tuple method found on internet

        state_tuple = tuple(tuple(row) for row in state) #we will use tuple because with np array this is too long or impossible
        return state_tuple


Let's test our class :

In [ ]:
initial_state = np.array([[1, 2, 3],
                          [4, 0, 5],
                          [6, 7, 8]])

goal_state = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 0]])

puzzle = EightPuzzle(initial_state, goal_state)
print("État initial :")
print(puzzle.initial_state)
possible_moves = puzzle.move_possible(puzzle.initial_state)
print("\nMouvements possibles :", possible_moves)
new_state = puzzle.move(puzzle.initial_state, "down")
print("\nÉtat après mouvement 'down' :")
print(new_state)
new_state = puzzle.move(new_state, "right")
print("\nÉtat après mouvement 'right' :")
print(new_state)
is_goal = puzzle.is_goal(new_state)
print("\nEst-ce l'état but ?", is_goal)

État initial :
[[1 2 3]
 [4 0 5]
 [6 7 8]]

Mouvements possibles : ['up', 'down', 'left', 'right']

État après mouvement 'down' :
[[1 2 3]
 [4 7 5]
 [6 0 8]]

État après mouvement 'right' :
[[1 2 3]
 [4 7 5]
 [6 8 0]]

Est-ce l'état but ? False


Our class is working so let's keep going

### 2. Uniform Cost Search (UCS) :


In [ ]:
%%add_to EightPuzzle
#adding the method to our class eightpuzzle
def uniform_cost_search(self):
    frontier = [(0, self.initial_state, [])] #creating our frontier based on state, nodes explored
    explored = set()
    while frontier:
        cost, current_state, path = frontier.pop(0)  #get rid of first element
        if self.is_goal(current_state):#checking goal
            return path, cost,len(explored)
        state_tuple = self.state_to_tuple(current_state)#changing to tuple to make it easier for the following
        explored.add(state_tuple)
        for action in self.move_possible(current_state): #here we enhance the move
            new_state = self.move(current_state, action)
            new_cost = cost + 1
            new_state_tuple = self.state_to_tuple(new_state)
            if new_state_tuple not in explored: #cheking if the state we're in is explored or not
                frontier.append((new_cost, new_state, path + [action]))

    return None, float('inf'),len(explored) # if no solution is found


In [ ]:
path, cost,nodes_exp = puzzle.uniform_cost_search()

print("Chemin trouvé :", path)
print("Coût :", cost)


Chemin trouvé : ['right', 'down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
Coût : 14


### 3. Heuristics for A* Search

Misplaced Tiles heuristic

In [ ]:
%%add_to EightPuzzle
def misplaced_tiles_heuristic(self, state): #basic method to get the sum of all missing tiles
    count = 0
    for i in range(self.taille):
        for j in range(self.taille):
            if state[i, j] != self.goal_state[i, j]:
                count += 1
    return count


In [ ]:
%%add_to EightPuzzle
def manhattan_distance_heuristic(self, state): #here we have to get the manhatthan distance for each tiles, this result in abs(i - goal_i) + abs(j - goal_j)
    distance = 0
    for i in range(self.taille):
        for j in range(self.taille):
            value = state[i, j]
            if value != 0: #checking if we are on the 0 spot thus we dont count if its the 0
                goal_i, goal_j = np.argwhere(self.goal_state == value)[0]
                distance += abs(i - goal_i) + abs(j - goal_j)
    return distance

In [ ]:
#lets test our two heuristics :
misplaced_tiles = puzzle.misplaced_tiles_heuristic(initial_state)
manhattan_distance = puzzle.manhattan_distance_heuristic(initial_state)

print(f"Misplaced Tiles Heuristic: {misplaced_tiles}")
print(f"Manhattan Distance Heuristic: {manhattan_distance}")

Misplaced Tiles Heuristic: 5
Manhattan Distance Heuristic: 6


###  4. Best-First Search Algorithm

In [ ]:
%%add_to EightPuzzle
def best_first_search(self, heuristic):
    queue = [(heuristic(self.initial_state), self.initial_state, [])]#h1 ouh2, state ici l'initial mais changera dans la boucle, chemin
    explored=set() #we use set to avoid getting back the same state so we dont go back on a thing we allready checked

    while queue : #on verifie que la frontiere ne soit pas vide vu quon pop chaque state a chaque iteration.
        queue.sort(key=lambda x : x[0])
        _, current_state, path = queue.pop(0) #we dont need heuristic back
        if self.is_goal(current_state):
           return path,len(explored)
        state_tuple = self.state_to_tuple(current_state)#je met en tuple pour faciliter le tri et eviter l'inf
        explored.add(state_tuple)

        for action in self.move_possible(current_state): #here solution is not goal so we need to get the moves possible and make it move
            new_state = self.move(current_state, action)  # Move to new state
            new_state_tuple = self.state_to_tuple(new_state)
            if new_state_tuple not in explored:
                new_path = path + [action] #we add the action to our old path
                new_heuristic = heuristic(self.move(current_state, action))
                queue.append((new_heuristic, self.move(current_state, action), new_path))#we add to queue our new path that we hot before
    return None,float('inf')


lets test our BFS method.

In [ ]:
heuristic=puzzle.manhattan_distance_heuristic(initial_state)
print("heuristic:",heuristic)
path, length = puzzle.best_first_search(puzzle.manhattan_distance_heuristic)
print("with manathan dist heur we have path :",path)
print("nodes explored :",length)
path, length = puzzle.best_first_search(puzzle.misplaced_tiles_heuristic)
heuristic=puzzle.misplaced_tiles_heuristic(initial_state)
print("heuristic:",heuristic)
print("with misplaced tiles heur we have path :",path)
print("nodes explored :",length)


heuristic: 6
with manathan dist heur we have path : ['left', 'down', 'right', 'up', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'right']
nodes explored : 95
heuristic: 5
with misplaced tiles heur we have path : ['right', 'down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
nodes explored : 151


### 5. Admissible Heuristic

In [ ]:
%%add_to EightPuzzle
def row_colon_heuristic(self,state): #here we do the same as for the misplaced tiles heuristic but we add the fact that the column and the rows are separated
    tiles_row=0
    tiles_col=0
    for i in range(self.taille):
        for j in range(self.taille):
            if state[i,j]!=0:
                goal_i,goal_j=np.argwhere(self.goal_state==state[i,j])[0] #getting the tiles in the place that we need (goal state )
                if i != goal_i:
                    tiles_row+=1
                if goal_j!=j:
                    tiles_col+=1
    return tiles_row+tiles_col


Lets test our last heuristic with BFS algorithm

In [ ]:
heuristic=puzzle.row_colon_heuristic(initial_state)
print("heuristic:",heuristic)
path, length = puzzle.best_first_search(puzzle.row_colon_heuristic)
print("with manathan dist heur we have path :",path)
print("nodes explored :",length)

heuristic: 5
with manathan dist heur we have path : ['left', 'down', 'right', 'up', 'left', 'down', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'right']
nodes explored : 93


Now we must compare our heuristic on an other intial state to see wich one is the best


In [ ]:
initial_state = np.array([[8, 4, 3],
                          [7, 0, 2],
                          [6, 5, 1]]) #changed initial state( more complicate)

goal_state = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 0]])
puzzle = EightPuzzle(initial_state, goal_state)
heuristic=puzzle.manhattan_distance_heuristic(initial_state)
print("heuristic manhattan:",heuristic)
heuristic=puzzle.row_colon_heuristic(initial_state)
print("heuristic row column :",heuristic)
heuristic=puzzle.misplaced_tiles_heuristic(initial_state)
print("heuristic misplaced :",heuristic)


heuristic manhattan: 16
heuristic row column : 12
heuristic misplaced : 8


This heuristic is admissible because it doesnt overestimate the path cost. Moreover this heuristic is slightly better than the first one misplace tiles because it takes in consideration the fact that this is a matrix with row and column. To finish, H3 row and column is not the best one, h2 manhathan distance is the best one because it gives exactly the number of moves needed to have the goal state.

### 6. A* Search

In [ ]:
%%add_to EightPuzzle
def a_star_search(self, heuristic):
    queue = [(0 + heuristic(self.initial_state), 0, self.initial_state, [])] #making a queue as a frontier of the heuristic f_n,  the cost g_n, the state and the path.
    explored=set()
    while queue : #we do it until the queue is null or the state is the goal state
        queue.sort(key=lambda x : x[0]) #wwe sort it to have least heuristic in the first place.
        f_n, g_n, current_state, path = queue.pop(0)
        if self.is_goal(current_state):
           return path,g_n,len(explored)
        state_tuple = self.state_to_tuple(current_state)
        explored.add(state_tuple)
        for action in self.move_possible(current_state):
            new_state = self.move(current_state, action)  # moving to new state
            new_state_tuple = self.state_to_tuple(new_state)
            if new_state_tuple not in explored:
                new_g_n = g_n + 1  # increment g_n by 1 (cost)
                new_f_n = new_g_n + heuristic(new_state)  # f(n) = g(n) + h(n)
                queue.append((new_f_n, new_g_n, new_state, path + [action]))

    return None, 'inf', len(explored)


In [ ]:
path,g_n, length = puzzle.a_star_search(puzzle.manhattan_distance_heuristic)
print("with manathan dist heur we have path :",path)
print("cost :",g_n)
print("nodes explored :",length)
path,g_n, length = puzzle.a_star_search(puzzle.misplaced_tiles_heuristic)
print("with misplaced tiles heur we have path :",path)
print("cost :",g_n)
print("nodes explored :",length)

with manathan dist heur we have path : ['right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down', 'down', 'left', 'up', 'right', 'down']
cost : 22
nodes explored : 805
with misplaced tiles heur we have path : ['right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'down', 'down', 'left', 'up', 'right', 'down']
cost : 22
nodes explored : 9853


We can see that for both heuristics the cost is the same but for the manhattan (h2) the number of nodes explored is way less than h1

### 7. Comparaison and analysis

Lets first have a look about the time of execution, the cost and the number of nodes explored :

In [ ]:
import time

start_time_ucs = time.time() #starting timer
path_ucs, cost_ucs ,nodes_exp= puzzle.uniform_cost_search()
end_time_ucs = time.time()#ending timer
execution_time_ucs = end_time_ucs - start_time_ucs
print(f"Uniform Cost Search  Execution Time: {execution_time_ucs} seconds")

start_time_a_star = time.time()
path_a_star, cost_a_star, nodes_explored = puzzle.a_star_search(puzzle.manhattan_distance_heuristic)
end_time_a_star = time.time()
execution_time_a_star = end_time_a_star - start_time_a_star
print(f"A* Search (Manhattan Heuristic) Execution Time: {execution_time_a_star} seconds")


Uniform Cost Search  Execution Time: 23.06769323348999 seconds
A* Search (Manhattan Heuristic) Execution Time: 0.1948690414428711 seconds


Let's store all the datas in a data frame so we can compare both


In [ ]:
import pandas as pd

results = {
    "Algorithm": ["Uniform Cost Search", "A* with Manhattan Distance"],
    "Cost (Solution)": [cost_ucs, cost_a_star],
    "Execution Time (seconds)": [execution_time_ucs, execution_time_a_star],
    "Nodes Explored": [nodes_exp, nodes_explored],
    "Path Length": [len(path_ucs) if path_ucs else 0, len(path_a_star) if path_a_star else 0]

}

df_results = pd.DataFrame(results)
df_results.head()


,Algorithm,Cost (Solution),Execution Time (seconds),Nodes Explored,Path Length
0,Uniform Cost Search,22,23.067693,106202,22
1,A* with Manhattan Distance,22,0.194869,805,22


The results show a clear difference in performance between Uniform Cost Search (UCS) and A* with the Manhattan Distance heuristic. Both algorithms find the optimal solution with a cost of 22 moves, but UCS takes 23.06 seconds and explores 106,202 nodes, while A* is much faster, taking only 0.19 seconds and exploring 805 nodes. This highlights the efficiency of the Manhattan heuristic in A*, which significantly reduces both the number of explored nodes and execution time while still providing the optimal solution.